# S02: データ変換（HTML → Parquet）

HTMLをパースしてParquet形式で保存・GitHubにプッシュ

**実行環境**: Sagemaker

---
## 1. 環境設定

In [ ]:
%pip install pyarrow -q

In [ ]:
import sys
import os
from pathlib import Path
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# プロジェクトルート
PROJECT_ROOT = Path('../../')
sys.path.insert(0, str(PROJECT_ROOT / 'src'))

# データパス
HTML_DIR = PROJECT_ROOT / 'data' / 'raceHTML'
PROCESSED_DIR = PROJECT_ROOT / 'data' / 'processed'
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

RACES_PARQUET = PROCESSED_DIR / 'races.parquet'
RESULTS_PARQUET = PROCESSED_DIR / 'results.parquet'

print(f"HTML_DIR: {HTML_DIR.resolve()}")
print(f"出力先: {PROCESSED_DIR.resolve()}")

In [ ]:
from scraper.parser import parse_multiple_html_full, parse_incremental_parquet
print("✅ モジュール読み込み完了")

---
## 2. パース設定

- **MODE = 'incremental'**: 新規HTMLのみパース（推奨）
- **MODE = 'full'**: 全件再パース

### limit_per_year
- **開発中**: 100（各年100件）
- **本番**: None（全件処理）

In [ ]:
# ========== 設定 ==========

# パースモード
MODE = 'incremental'  # 'incremental' or 'full'

# 件数制限（Noneで無制限）
LIMIT_PER_YEAR = None  # 開発時: 100, 本番: None

print(f"モード: {MODE}")
print(f"制限: {LIMIT_PER_YEAR if LIMIT_PER_YEAR else '無制限'}")

if RACES_PARQUET.exists():
    existing = pd.read_parquet(RACES_PARQUET)
    print(f"既存データ: {len(existing):,} レース")
else:
    print("既存データ: なし（初回実行）")

---
## 3. HTMLパース実行

In [ ]:
def progress_callback(processed, total, skipped=0):
    pct = processed / total * 100 if total > 0 else 0
    print(f"\r進捗: {processed:,}/{total:,} ({pct:.1f}%) スキップ: {skipped:,}", end='')

if MODE == 'incremental':
    # 差分更新モード
    print("📊 差分更新モード: 新規HTMLのみパース")
    new_races, new_horses = parse_incremental_parquet(
        HTML_DIR,
        RACES_PARQUET,
        RESULTS_PARQUET,
        years=None,
        limit_per_year=LIMIT_PER_YEAR,
        progress_callback=progress_callback
    )
else:
    # 全件再パース
    print("📊 全件再パースモード")
    if LIMIT_PER_YEAR:
        print(f"   ⚠️ 開発モード: 各年 {LIMIT_PER_YEAR} 件まで")
    
    race_list, horse_list = parse_multiple_html_full(
        HTML_DIR,
        years=None,
        progress_callback=progress_callback,
        limit_per_year=LIMIT_PER_YEAR
    )
    
    races_df = pd.DataFrame(race_list)
    results_df = pd.DataFrame(horse_list)
    
    races_df.to_parquet(RACES_PARQUET, index=False, compression='snappy')
    results_df.to_parquet(RESULTS_PARQUET, index=False, compression='snappy')
    
    print(f"\n\n✅ パース完了")
    print(f"   レース数: {len(race_list):,}")
    print(f"   出走馬データ数: {len(horse_list):,}")

---
## 4. データ確認

In [ ]:
races_df = pd.read_parquet(RACES_PARQUET)
results_df = pd.read_parquet(RESULTS_PARQUET)

print("=== データ統計 ===")
print(f"レース数: {len(races_df):,}")
print(f"出走馬データ数: {len(results_df):,}")
print(f"\nファイルサイズ:")
print(f"  races.parquet: {RACES_PARQUET.stat().st_size / 1024 / 1024:.2f} MB")
print(f"  results.parquet: {RESULTS_PARQUET.stat().st_size / 1024 / 1024:.2f} MB")

if 'year' in races_df.columns:
    print("\n=== 年別レース数 ===")
    print(races_df.groupby('year').size().to_string())

In [ ]:
print("=== レースデータ サンプル ===")
display(races_df.tail(5))

In [ ]:
print("=== 出走馬データ サンプル ===")
display(results_df.tail(5))

---
## 5. GitHubにプッシュ

In [ ]:
import subprocess
import os

os.chdir(PROJECT_ROOT)

print("📤 GitHubにプッシュ中...")

try:
    subprocess.run(['git', 'add', 'data/processed/races.parquet', 'data/processed/results.parquet'], check=True)
    
    result = subprocess.run(
        ['git', 'commit', '-m', 'Update race data parquet files'],
        capture_output=True, text=True
    )
    if result.returncode == 0:
        print("✅ コミット完了")
    elif 'nothing to commit' in result.stdout + result.stderr:
        print("ℹ️ 変更なし")
    else:
        print(f"コミット: {result.stderr}")
    
    result = subprocess.run(['git', 'push'], capture_output=True, text=True)
    if result.returncode == 0:
        print("✅ プッシュ完了!")
        print("\n🌐 オンラインURL:")
        print("https://raw.githubusercontent.com/iinumac/keiba_prediction/main/data/processed/races.parquet")
    else:
        print(f"プッシュ: {result.stderr}")
        
except Exception as e:
    print(f"❌ エラー: {e}")